In [6]:
import mysql.connector
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib


In [7]:
conn = mysql.connector.connect(host = "localhost", password = "123456", user = "root")
mycursor = conn.cursor()

str_select_all_movieinformation = "SELECT * FROM `finaldataimg`.`movieinformation`;" 
mycursor.execute(str_select_all_movieinformation)
select_all_movieinformation = mycursor.fetchall() 

all_movie_data = []

if select_all_movieinformation:
    index = 1
    for movieinformation in select_all_movieinformation:
        # movie_id
        # print("Movie ID: " + str(movieinformation[0]))

        # movie_name
        # print("Movie name: " + movieinformation[1])

        # describe
        # print("Describe: " + movieinformation[8])

        # genres
        genres_list = []
        str_select_genres_id_movie_genres = "SELECT * FROM `finaldataimg`.`movie_genres` WHERE `movie_id` = " + str(movieinformation[0]) + ";"
        mycursor.execute(str_select_genres_id_movie_genres)
        select_genres_id_movie_genres = mycursor.fetchall()

        if select_genres_id_movie_genres:
            for movie_genres in select_genres_id_movie_genres:
                str_select_all_genres_genres = "SELECT * FROM `finaldataimg`.`genres` WHERE `genres_id` = " + str(movie_genres[1]) + ";"
                mycursor.execute(str_select_all_genres_genres)
                select_all_genres_genres = mycursor.fetchall()

                if select_all_genres_genres:
                    for genres in select_all_genres_genres:
                        if genres[1] == "":
                            continue
                        # print("Genres: " + genres[1])
                        genres_list.append(genres[1])

        # director
        director_list = []
        str_select_director_id_movie_director = "SELECT * FROM `finaldataimg`.`movie_director` WHERE `movie_id` = " + str(movieinformation[0]) + ";"
        mycursor.execute(str_select_director_id_movie_director)
        select_director_id_movie_director = mycursor.fetchall()

        if select_director_id_movie_director:
            for movie_director in select_director_id_movie_director:
                str_select_all_director_director = "SELECT * FROM `finaldataimg`.`director` WHERE `director_id` = " + str(movie_director[1]) + ";"
                mycursor.execute(str_select_all_director_director)
                select_all_director_director = mycursor.fetchall()

                if select_all_director_director:
                    for director in select_all_director_director:
                        if director[1] == "":
                            continue
                        # print("Director: " + director[1])
                        director_list.append(director[1])
                
        # cast
        cast_list = []
        str_select_cast_id_movie_cast = "SELECT * FROM `finaldataimg`.`movie_cast` WHERE `movie_id` = " + str(movieinformation[0]) + ";"
        mycursor.execute(str_select_cast_id_movie_cast)
        select_cast_id_movie_cast = mycursor.fetchall()

        if select_cast_id_movie_cast:
            for movie_cast in select_cast_id_movie_cast:
                if len(cast_list) == 15:
                    break

                str_select_all_cast_cast = "SELECT * FROM `finaldataimg`.`cast` WHERE `cast_id` = " + str(movie_cast[1]) + ";"
                mycursor.execute(str_select_all_cast_cast)
                select_all_cast_cast = mycursor.fetchall()
                

                if select_all_cast_cast:
                    for cast in select_all_cast_cast:
                        if cast[1] == "":
                            continue
                        # print("Cast: " + cast[1])
                        cast_list.append(cast[1])

                                
        movie_data = {
            "movie_id": movieinformation[0],
            "movie_name": movieinformation[1],
            "describe": movieinformation[8],
            "genres": genres_list,
            "director": director_list,
            "cast": cast_list
        }

        # print(movie_data)
        all_movie_data.append(movie_data)
        # print("--------------------------------------------------------------------------------------------------------------------------------")
        # if index == 1:
        #     break
        
        index += 1

In [8]:
import numpy as np
movie_df = pd.DataFrame(all_movie_data)

# Xử lý dữ liệu
movie_df['genres'] = movie_df['genres'].apply(lambda x: ' '.join(x))
movie_df['cast'] = movie_df['cast'].apply(lambda x: ' '.join(x))
movie_df['director'] = movie_df['director'].apply(lambda x: ' '.join(x))

# Loại bỏ các hàng có giá trị NaN
movie_df.dropna(inplace=True)

# Tạo TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Tạo ma trận TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(movie_df['movie_name'] + ' ' + movie_df['genres'] + ' ' + movie_df['genres'] + ' ' +  movie_df['genres'] + ' ' +  movie_df['genres'] + ' ' +  movie_df['describe'] + ' ' + movie_df['cast'] + ' ' + movie_df['director'])
# tfidf_matrix = tfidf_vectorizer.fit_transform(movie_df['movie_name'])
# tfidf_matrix = tfidf_vectorizer.fit_transform(movie_df['genres'])
# tfidf_matrix = tfidf_vectorizer.fit_transform(movie_df['describe'])
# tfidf_matrix = tfidf_vectorizer.fit_transform(movie_df['cast'])
# tfidf_matrix = tfidf_vectorizer.fit_transform(movie_df['director'])
# Tính toán độ tương đồng cosine
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

ind_movie = 134
# print("Movie id: ", movie_df.iloc[ind_movie]['movie_id'])
# print("Movie name: ", movie_df.iloc[ind_movie]['movie_name'])
# print("Describe: ", movie_df.iloc[ind_movie]['describe'])
# print("Genres: ", movie_df.iloc[ind_movie]['genres'])
# print("Director: ", movie_df.iloc[ind_movie]['director'])
# print("Cast: ", movie_df.iloc[ind_movie]['cast'])

second_max_cosine_index = np.argsort(cosine_sim[ind_movie])[-2]
# # print(cosine_sim[ind_movie][second_max_cosine_index])

# print("Movie id result: ")
# print(movie_df.iloc[second_max_cosine_index]['movie_id'], cosine_sim[ind_movie][second_max_cosine_index])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-3]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-3]])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-4]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-4]])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-5]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-5]])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-6]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-6]])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-7]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-7]])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-8]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-8]])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-9]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-9]])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-10]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-10]])
# print(movie_df.iloc[np.argsort(cosine_sim[ind_movie])[-11]]['movie_id'], cosine_sim[ind_movie][np.argsort(cosine_sim[ind_movie])[-11]])

print("Movie name: ", movie_df.iloc[second_max_cosine_index]['movie_name'])
# print("Describe: ", movie_df.iloc[second_max_cosine_index]['describe'])
# print("Genres: ", movie_df.iloc[second_max_cosine_index]['genres'])
# print("Director: ", movie_df.iloc[second_max_cosine_index]['director'])
# print("Cast: ", movie_df.iloc[second_max_cosine_index]['cast'])

# Lưu các đối tượng cần thiết
joblib.dump(tfidf_vectorizer, 'model/tfidf_vectorizer.pkl')
joblib.dump(tfidf_matrix, 'model/tfidf_matrix.pkl')
joblib.dump(movie_df, 'model/movie_df.pkl')
joblib.dump(cosine_sim, 'model/cosine_sim.pkl')

Movie name:  Fast X: Part 2


['model/cosine_sim.pkl']